In [1]:
import numpy as np
import pandas as pd

In [22]:
raw_df = pd.read_csv('../../data/raw_data.csv')

In [ ]:
raw_df[((raw_df['sma_03z2'] > 3.0) & (raw_df['sma_12z2'] > 2.0) & (raw_df['sma_37z1'] > 3.0) & (raw_df['sma_01z1']==1))]
# 일반, 그외, 전자담배를 모두 피워본 적 없고 액상형을 피워본 사람 -> 존재 x
# 즉, 액상형만 피워왔던 row는 raw_data에서도 표본없음

,EXAMIN_YEAR,exmprs_no,age,sex,CTPRVN_CODE,PBHLTH_CODE,SPOT_NO,HSHLD_CODE,MBHLD_CODE,DONG_TY_CODE,...,cpr_02z1,cpr_03a2,cpr_03b2,cpr_04z1,sob_01z1,sob_02z1,soa_01z1,soa_06z2,soa_07z1,sod_02z3


In [42]:
df = pd.read_csv('../../data/analy_data.csv')

In [43]:
df[["sma_08z1"]].value_counts()#, "sma_11z2"]]

# 9 모름 -> 2 아니오로 처리
df.loc[df["sma_08z1"] == 9, 'sma_08z1'] = 2.0

In [45]:
df[['sma_08z1', 'churn']].value_counts()

sma_08z1  churn
2.0       1        46103
          0        33278
1.0       0         7293
          1         3148
Name: count, dtype: int64

In [ ]:
df[df['churn']==1][['sma_08z1', 'sma_11z2', 'churn']].value_counts()
# 금연 성공자라고 지정한 row 중 한 달 이내 액상형을 며칠 피웠는지 값에 대한 개수 출력

# 액상형을 피워봤고(1), 금연자(1)인 row만 문제됨. 모든 담배의 금연에 성공했을지 미지수이므로.
# (1) 액상형을 피워봤으면 모두 churn을 0으로 변경 -> 액상형을 피워봤으면 무조건 금연실패로 간주하는 것과 같음. -> X
# (2) 애매한 해당 row만 제거 -> churn 정확도는 확실해짐 -> Liquid_method1() 함수화
# (3) 액상형 피워봤고, 최근 한 달 동안 피운 일수가 0을 초과하면 churn을 0으로 변경 -> Liquid_method2() 함수화
#     단, 금연 기준 '30일'이 다른 피쳐에도 적용될 수 있는가에 대한 문제. 설문에서 흡연자에게 '최근 한 달 동안'을 물으므로 명분 있음

sma_08z1  sma_11z2  churn
2.0       88.0      1        46100
1.0       0.0       1         1762
          30.0      1          615
          31.0      1          461
          10.0      1           48
          15.0      1           48
          20.0      1           45
          1.0       1           44
          5.0       1           28
          3.0       1           20
          2.0       1           20
          7.0       1           19
          25.0      1           15
          4.0       1            7
          14.0      1            6
2.0       99.0      1            3
1.0       6.0       1            2
          12.0      1            2
          28.0      1            2
          22.0      1            1
          8.0       1            1
          29.0      1            1
          27.0      1            1
Name: count, dtype: int64

In [ ]:
# 액상형을 피워봤는데 churn이 1이라 애매한 row 모두 제거. 3,148건 제거됨
def Liquid_method1(df_merge):
    # 9 모름 -> 2 아니오로 처리
    df_merge.loc[df_merge["sma_08z1"] == 9, 'sma_08z1'] = 2.0
    df_merge = df_merge[~((df_merge['sma_08z1']==1) & (df_merge['churn']==1))]
    return df_merge


# 액상형 피워봤고, 최근 한 달 동안 피운 일수가 0을 초과하는데 churn이 1이면 0으로 변경
def Liquid_method2(df_merge):
    # 9 모름 -> 2 아니오로 처리
    df_merge.loc[df_merge["sma_08z1"] == 9, 'sma_08z1'] = 2.0
    df_merge.loc[(df_merge['sma_08z1'] == 1) & (df_merge['sma_11z2'] != 0) & (df_merge['churn'] == 1), 'churn'] = 0
    return df_merge

In [33]:
df_test1 = Liquid_method1(df)
df_test2 = Liquid_method2(df)

In [ ]:
df_test1[['sma_08z1', 'churn']].value_counts() # (1, 1)인 row 모조리 제거했을 때

sma_08z1  churn
2.0       1        46103
          0        33278
1.0       0         7293
Name: count, dtype: int64

In [ ]:
df_test2[['sma_08z1', 'sma_11z2', 'churn']].value_counts() # 조건에 부합하는 churn만 0으로 바꿨을 때

sma_08z1  sma_11z2  churn
2.0       88.0      1        46100
                    0        33276
1.0       0.0       0         3718
          30.0      0         1813
          0.0       1         1762
          31.0      0          984
          10.0      0          321
          1.0       0          319
          20.0      0          265
          15.0      0          250
          2.0       0          224
          5.0       0          203
          3.0       0          175
          7.0       0          161
          25.0      0           71
          4.0       0           62
          14.0      0           33
          8.0       0           16
          12.0      0           12
          6.0       0           11
          28.0      0            8
          99.0      0            8
          18.0      0            4
          21.0      0            3
2.0       99.0      1            3
1.0       22.0      0            2
          9.0       0            2
          26.0      0        

In [ ]:
print(df_test1['churn'].value_counts(), df_test2['churn'].value_counts()) # 각 df의 churn 데이터 개수

churn
1    46103
0    40571
Name: count, dtype: int64 churn
1    47865
0    41957
Name: count, dtype: int64
